In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage.io import imread, imshow
%matplotlib inline

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
from subprocess import check_output
print(check_output(['ls', './data/train']).decode('utf8'))

Type_1
Type_2
Type_3



# retrieve all images

In [6]:
from glob import glob
basepath = './data/train/'

all_cervix_images = []
for path in sorted(glob(basepath + '*')):
    cervix_type = path.split('/')[-1]
    cervix_images = glob(basepath + cervix_type + '/*')
    all_cervix_images += cervix_images

# generate dataframe [path, type]
all_cervix_images = pd.DataFrame({'path': all_cervix_images})
all_cervix_images['type'] = all_cervix_images['path'].apply(lambda path : path.split('/')[-2])

In [7]:
def get_images_and_types(df):
    '''
    df[path, type] -> all_images, all_image_types
    '''
    all_images = []
    all_image_types = []
    for t in df['type'].unique():
        sample_cnt = 0
        for _, row in df[df['type'] == t].iterrows():
            try:
                img = imread(row.path)
                all_images.append(img)
                all_image_types.append(row.type)
                sample_cnt += 1
            except:
                print('image read failed for {}'.format(row.path))
            if sample_cnt > 100:
                break
    return all_images, all_image_types
        

In [8]:
all_images, all_image_types = get_images_and_types(all_cervix_images)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 393216 bytes but only got 0. Skipping tag 3
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33554432 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 25165824 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: U

image read failed for ./data/train/Type_1/1339.jpg


/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 692191232 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 219021312 bytes but only got 13904. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20791909992 bytes but only got 0. Skipping tag 39940
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 619577344 bytes but only got 0. Skipping tag 4
  "Skipping tag %s" % (size, len(data), tag))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/PIL/TiffIma

# transform images for CNN inputs

In [9]:
def transform_image(img, resized_dim, to_gray=False):
    '''
    Return transformed image for inputs of CNN.
    1. resize to (usually )smaller dimension
    2. to gray (if the param is True)
    3. normalize
    4. reshape and norm
    '''
#    1. resize 
    resized = cv2.resize(img, (resized_dim, resized_dim), cv2.INTER_LINEAR)

#     2. to gray
    if to_gray:
        resized = cv2.cvtColor(resized, cv2.COLOR_RGB2GRAY).astype('float')
    else:
        resized = resized.astype('float')
#     3. normalize
    normalized = cv2.normalize(resized, None, 0.0, 1.0, cv2.NORM_MINMAX)
    
    return normalized
#     4. reshape -> not necessary? for keras inputs
#     timg = normalized.reshape(1, np.prod(normalized.shape))
    
#     return timg/np.linalg.norm(timg)
            



In [10]:
# gray_all_images_as_vecs = [transform_image(img, 100) for img in all_images]
# gray_imgs_mat = np.array(gray_all_images_as_vecs).squeeze()
gray_imgs_mat = [transform_image(img, 100) for img in all_images]
gray_imgs_mat = np.array(gray_imgs_mat)
gray_image_types = np.array(all_image_types)
gray_imgs_mat.shape,gray_image_types.shape


((303, 100, 100, 3), (303,))

# split train data and test data

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
y = LabelEncoder().fit_transform(all_image_types)
X = gray_imgs_mat
X.shape, y.shape

((303, 100, 100, 3), (303,))

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((203, 100, 100, 3), (100, 100, 100, 3), (203, 3), (100, 3))

# define CNN

In [13]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(100, 100, 3)))

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(100, 100,..., activation="relu")`
  from ipykernel import kernelapp as app


In [14]:
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  if __name__ == '__main__':


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, verbose=1)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
203/203 [==============================] - 4s - loss: 2.9459 - acc: 0.3103     
Epoch 2/10
203/203 [==============================] - 0s - loss: 1.0987 - acc: 0.4236     
Epoch 3/10
203/203 [==============================] - 0s - loss: 1.0768 - acc: 0.4236     
Epoch 4/10
203/203 [==============================] - 0s - loss: 1.0080 - acc: 0.5419     
Epoch 5/10
203/203 [==============================] - 0s - loss: 0.9483 - acc: 0.5665     
Epoch 6/10
203/203 [==============================] - 0s - loss: 0.9002 - acc: 0.6355     
Epoch 7/10
203/203 [==============================] - 0s - loss: 0.7996 - acc: 0.7044     
Epoch 8/10
203/203 [==============================] - 0s - loss: 0.6885 - acc: 0.7438     
Epoch 9/10
203/203 [==============================] - 0s - loss: 0.5424 - acc: 0.8227     
Epoch 10/10
203/203 [==============================] - 0s - loss: 0.5058 - acc: 0.8473     


# evaluate model on test data

In [17]:
score = model.evaluate(X_test, Y_test, verbose=0)

In [18]:
score

[1.1624606800079347, 0.40999999999999998]

In [90]:
model.metrics_names

['loss', 'acc']